<a href="https://colab.research.google.com/github/JieRou-1007/FYP/blob/master/latest_latest_load_and_save_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/pytorch/examples

Cloning into 'examples'...
remote: Enumerating objects: 2805, done.
remote: Total 2805 (delta 0), reused 0 (delta 0), pack-reused 2805
Receiving objects: 100% (2805/2805), 39.16 MiB | 22.56 MiB/s, done.
Resolving deltas: 100% (1413/1413), done.


In [ ]:
 from google.colab import drive
 drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [ ]:
!unzip '/content/gdrive/MyDrive/fyp/train 256/1/train.zip' -d '/content/train/1'

Archive:  /content/gdrive/MyDrive/fyp/train 256/1/train.zip
  inflating: /content/train/1/train/00000000.jpg  
  inflating: /content/train/1/train/00000000_(2).jpg  
  inflating: /content/train/1/train/00000000_(3).jpg  
  inflating: /content/train/1/train/00000000_(4).jpg  
  inflating: /content/train/1/train/00000000_(5).jpg  
  inflating: /content/train/1/train/00000000_(6).jpg  
  inflating: /content/train/1/train/00000000_(7).jpg  
  inflating: /content/train/1/train/00000001.jpg  
  inflating: /content/train/1/train/00000001_(2).jpg  
  inflating: /content/train/1/train/00000001_(3).jpg  
  inflating: /content/train/1/train/00000001_(4).jpg  
  inflating: /content/train/1/train/00000001_(5).jpg  
  inflating: /content/train/1/train/00000001_(6).jpg  
  inflating: /content/train/1/train/00000001_(7).jpg  
  inflating: /content/train/1/train/00000002.jpg  
  inflating: /content/train/1/train/00000002_(2).jpg  
  inflating: /content/train/1/train/00000002_(3).jpg  
  inflating: /con

In [ ]:
!unzip '/content/gdrive/MyDrive/fyp/test 256/1/test.zip' -d '/content/test'

Archive:  /content/gdrive/MyDrive/fyp/test 256/1/test.zip
  inflating: /content/test/test/00000573_(4).jpg  
  inflating: /content/test/test/00000574.jpg  
  inflating: /content/test/test/00000574_(2).jpg  
  inflating: /content/test/test/00000574_(3).jpg  
  inflating: /content/test/test/00000574_(4).jpg  
  inflating: /content/test/test/00000575.jpg  
  inflating: /content/test/test/00000575_(2).jpg  
  inflating: /content/test/test/00000575_(3).jpg  
  inflating: /content/test/test/00000575_(4).jpg  
  inflating: /content/test/test/00000576.jpg  
  inflating: /content/test/test/00000576_(2).jpg  
  inflating: /content/test/test/00000576_(3).jpg  
  inflating: /content/test/test/00000576_(4).jpg  
  inflating: /content/test/test/00000577.jpg  
  inflating: /content/test/test/00000577_(2).jpg  
  inflating: /content/test/test/00000577_(3).jpg  
  inflating: /content/test/test/00000577_(4).jpg  
  inflating: /content/test/test/00000578.jpg  
  inflating: /content/test/test/00000578_(2)

In [ ]:
pip install -r /content/examples/vae/requirements.txt

In [ ]:
def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min

In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import numpy as np
import shutil
import torchvision

# check if CUDA is available
use_cuda = torch.cuda.is_available()
parser = argparse.ArgumentParser(description='VAE MNIST Example')
batch_size = 128
n_epochs = 50
seed = 1 
log_interval = 10
# parser.add_argument('--batch-size', type=int, default=128, metavar='N',
#                     help='input batch size for training (default: 128)')
# parser.add_argument('--epochs', type=int, default=20, metavar='N',
#                     help='number of epochs to train (default: 10)')
# parser.add_argument('--no-cuda', action='store_true', default=False,
#                     help='disables CUDA training')
# parser.add_argument('--seed', type=int, default=1, metavar='S',
#                     help='random seed (default: 1)')
# parser.add_argument('--log-interval', type=int, default=10, metavar='N',
#                     help='how many batches to wait before logging training status')
# args = parser.parse_args()
# args.cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(seed)

device = torch.device("cuda")

kwargs = {'num_workers': 1, 'pin_memory': True} 
TRAIN_DATA_PATH = "/content/train"
TEST_DATA_PATH = "/content/test"
TRANSFORM_IMG = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(128),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406],
    #                      std=[0.229, 0.224, 0.225] )
    ])

train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=TRANSFORM_IMG)
test_data = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=TRANSFORM_IMG)


loaders = {
  'train': torch.utils.data.DataLoader(train_data, batch_size, shuffle=True, **kwargs),
  
  'test' : torch.utils.data.DataLoader(test_data, batch_size, shuffle=True, **kwargs),
}

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(3*128*128, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 3*128*128)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(x.size(0), 3*128*128))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(x.size(0), 3*128*128), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD


def train(start_epochs, n_epochs, test_loss_min_input, loaders, model, optimizer, criterion, use_cuda, checkpoint_path, best_model_path):
    test_loss_min = test_loss_min_input 

    for epoch in range(start_epochs, n_epochs + 1):
      model.train()
      train_loss = 0
      for batch_idx, (data, _) in enumerate(loaders['train']):
          # move to GPU
          if use_cuda:
              data, _ = data.cuda(), _.cuda()
          data = data.to(device)
          optimizer.zero_grad()
          recon_batch, mu, logvar = model(data)
          loss = loss_function(recon_batch, data, mu, logvar)
          loss.backward()
          train_loss += loss.item()
          optimizer.step()
          if batch_idx % log_interval == 0:
              print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch, batch_idx * len(data), len(loaders['train'].dataset),
                  100. * batch_idx / len(loaders['train']),
                  loss.item() / len(data)))

      print('====> Epoch: {} Average loss: {:.4f}'.format(
            epoch, train_loss / len(loaders['train'].dataset)))

      #test
      model.eval()
      test_loss = 0
      with torch.no_grad():
          for i, (data, _) in enumerate(loaders['test']):
              # move to GPU
              if use_cuda:
                  data, _ = data.cuda(), _.cuda()
                  data = data.to(device)
              recon_batch, mu, logvar = model(data)
              test_loss += loss_function(recon_batch, data, mu, logvar).item()
              if i == 0:
                  n = min(data.size(0), 8)
                  comparison = torch.cat([data[:n], recon_batch.view(128, 3, 128, 128)[:n]])
                  save_image(comparison.cpu(),
                          '/content/examples/vae/results/reconstruction_' + str(epoch) + '.png', nrow=n)
                  
      checkpoint = {
              'epoch': epoch + 1,
              'valid_loss_min': test_loss,
              'state_dict': model.state_dict(),
              'optimizer': optimizer.state_dict(),
          }

      checkpoint_path = './gdrive/MyDrive/ckp/cifar_net.pth'
      best_model_path = './gdrive/MyDrive/ckp/best.pth'
      # save checkpoint
      save_ckp(checkpoint, False, checkpoint_path, best_model_path)

      ## TODO: save the model if validation loss has decreased
      if test_loss <= test_loss_min:
          print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(test_loss_min,test_loss))
          # save checkpoint as best model
          save_ckp(checkpoint, True, checkpoint_path, best_model_path)
          test_loss_min = test_loss

      test_loss /= len(loaders['test'].dataset)
      print('====> Test set loss: {:.4f}'.format(test_loss))

      with torch.no_grad():
        sample = torch.randn(64, 20).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 3, 128, 128),
                    '/content/examples/vae/results/sample_' + str(epoch) + '.png')
    return model

trained_model = train(1, n_epochs, np.Inf,loaders, model, optimizer, loss_function,use_cuda, './cifar_net.pth', './best.pth')  
# PATH = './cifar_net.pth'
# torch.save(model.state_dict(), PATH)  

FileNotFoundError: ignored

In [ ]:
trained_model = train(1, n_epochs, np.Inf,loaders, model, optimizer, loss_function,use_cuda, './cifar_net.pth', './best.pth')

In [ ]:
class VAEtry(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(3*128*128, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 3*128*128)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(1.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(x.size(0), 3*128*128))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

NameError: ignored

In [ ]:
# load model
model = VAEtry()
if use_cuda:
    model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = loss_function
#criterion = loss_function
# define checkpoint saved path
ckp_path = "/content/gdrive/MyDrive/cifar_net50.pth"

In [ ]:
model, optimizer, start_epoch, valid_loss_min = load_ckp(ckp_path, model, optimizer)

In [ ]:
print("model = ", model)
print("optimizer = ", optimizer)
print("start_epoch = ", start_epoch)
print("valid_loss_min = ", valid_loss_min)
print("valid_loss_min = {:.6f}".format(valid_loss_min))

model =  VAE(
  (fc1): Linear(in_features=49152, out_features=400, bias=True)
  (fc21): Linear(in_features=400, out_features=20, bias=True)
  (fc22): Linear(in_features=400, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=400, bias=True)
  (fc4): Linear(in_features=400, out_features=49152, bias=True)
)
optimizer =  Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)
start_epoch =  3
valid_loss_min =  31613200.25
valid_loss_min = 31613200.250000


In [ ]:
trained_model = train(start_epoch, 100, valid_loss_min, loaders, model, optimizer, criterion, use_cuda, "/content/cifar_net.pth", "/content/best.pth")

Train Epoch: 51 [0/3300 (0%)]	Loss: 56706.902344
Train Epoch: 51 [1280/3300 (38%)]	Loss: 34184.289062
Train Epoch: 51 [2560/3300 (77%)]	Loss: 33127.425781
====> Epoch: 51 Average loss: 35784.2795
====> Test set loss: 32309.5992
Train Epoch: 52 [0/3300 (0%)]	Loss: 32740.355469
Train Epoch: 52 [1280/3300 (38%)]	Loss: 32673.179688
Train Epoch: 52 [2560/3300 (77%)]	Loss: 31688.294922
====> Epoch: 52 Average loss: 32199.0170
====> Test set loss: 31569.8865
Train Epoch: 53 [0/3300 (0%)]	Loss: 32094.380859
Train Epoch: 53 [1280/3300 (38%)]	Loss: 31496.214844
Train Epoch: 53 [2560/3300 (77%)]	Loss: 31647.292969
====> Epoch: 53 Average loss: 31771.2422
====> Test set loss: 31197.2825
Train Epoch: 54 [0/3300 (0%)]	Loss: 31610.089844
Train Epoch: 54 [1280/3300 (38%)]	Loss: 31904.277344
Train Epoch: 54 [2560/3300 (77%)]	Loss: 31026.011719
====> Epoch: 54 Average loss: 31364.5694
====> Test set loss: 30766.2948
Train Epoch: 55 [0/3300 (0%)]	Loss: 31295.865234
Train Epoch: 55 [1280/3300 (38%)]	Loss:

In [ ]:
!python /content/examples/vae/main.py

Files already downloaded and verified
Train Epoch: 1 [0/50000 (0%)]	Loss: 2152.270508
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2112.378662
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2061.184814
Train Epoch: 1 [3840/50000 (8%)]	Loss: 2030.074097
Train Epoch: 1 [5120/50000 (10%)]	Loss: 2028.063354
Train Epoch: 1 [6400/50000 (13%)]	Loss: 1987.913208
Train Epoch: 1 [7680/50000 (15%)]	Loss: 1976.817505
Train Epoch: 1 [8960/50000 (18%)]	Loss: 1968.977783
Train Epoch: 1 [10240/50000 (20%)]	Loss: 1984.599121
Train Epoch: 1 [11520/50000 (23%)]	Loss: 1967.367432
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1949.873535
Train Epoch: 1 [14080/50000 (28%)]	Loss: 1948.278809
Train Epoch: 1 [15360/50000 (31%)]	Loss: 1938.056030
Train Epoch: 1 [16640/50000 (33%)]	Loss: 1947.667114
Train Epoch: 1 [17920/50000 (36%)]	Loss: 1956.501709
Train Epoch: 1 [19200/50000 (38%)]	Loss: 1938.052612
Train Epoch: 1 [20480/50000 (41%)]	Loss: 1932.170654
Train Epoch: 1 [21760/50000 (43%)]	Loss: 1896.087036
Train Epoch: 1 [23040/5

In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(3*32*32, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 3*32*32)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(x.size(0), 3*32*32))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

device = torch.device("cuda")
model = VAE().to(device)
# model.load_state_dict(torch.load('/content/cifar_net.pth'))

In [ ]:
ckp_path = "/content/cifar_net2.pth"

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
model, optimizer, start_epoch, valid_loss_min = load_ckp(ckp_path, model, optimizer)

In [ ]:
print("model = ", model)
print("optimizer = ", optimizer)
print("start_epoch = ", start_epoch)
print("valid_loss_min = ", valid_loss_min)
print("valid_loss_min = {:.6f}".format(valid_loss_min))

model =  VAE(
  (fc1): Linear(in_features=3072, out_features=400, bias=True)
  (fc21): Linear(in_features=400, out_features=20, bias=True)
  (fc22): Linear(in_features=400, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=400, bias=True)
  (fc4): Linear(in_features=400, out_features=3072, bias=True)
)
optimizer =  Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)
start_epoch =  2
valid_loss_min =  1863.40787734375
valid_loss_min = 1863.407877


In [ ]:
trained_model = train(start_epoch, 6, valid_loss_min, loaders, model, optimizer, criterion, use_cuda, "./checkpoint/current_checkpoint.pt", "./best_model/best_model.pt")

NameError: ignored

In [ ]:
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=False, transform=transforms.ToTensor()),
    batch_size=128, shuffle=True)

In [ ]:
output = model(test_loader)

AttributeError: ignored

In [ ]:
device = torch.device("cuda")

with torch.no_grad():
        sample = torch.randn(64, 20).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1,32 , 32),
                    '/content/sample_' + str(epoch) + '.png')


RuntimeError: ignored

In [ ]:
model.eval()
test_loss = 0
with torch.no_grad():
    for i, (data, _) in enumerate(test_loader):
        data = data.to(device)
        recon_batch, mu, logvar = model(data)
      #  test_loss += loss_function(recon_batch, data, mu, logvar).item()
        if i == 0:
            n = min(data.size(0), 1)
            comparison = recon_batch.view(128, 3, 32, 32)[:n]
            save_image(comparison.cpu(),
                      '/content/try' + str(2) + '.png', nrow=n)